In [ ]:
%pip install yfinance 
%pip install duckdb
%pip install pandas 
import yfinance as yf
import duckdb
import pandas as pd
from datetime import datetime, timedelta, timezone



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 12.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### What the following code block is doing:

we use the `isinstance` function to say is our df multiindexed if it is then do the following:

- for each ticker we want to remove all whitespace with `.strip()` then if our ticker is included within the `columns.levels[0]`  reset the index for that ticker and change columns to lower case, add this to our array called `frames`

In [141]:
def fetch_prices(tickers, period_days):
    # Fetch daily adjusted OHLCV for the last N days
    start = (datetime.now(timezone.utc) - timedelta(days=period_days)).date().isoformat()
    # .strip() to handle any extra spaces in ticker list
    df = yf.download(
        tickers=[t.strip() for t in tickers],
        start=start,
        interval="1d",
        group_by="ticker",
        threads=True,
        progress=False,
    )
    # Normalize to long format: date, ticker, open, high, low, close, volume
    frames = []
    if isinstance(df.columns, pd.MultiIndex):
        for t in tickers:
            if t.strip() not in df.columns.levels[0]:
                continue
            sub = df[t.strip()].reset_index()
            sub = sub.rename(columns=lambda x: x.lower())
            sub["ticker"] = t.strip()
            frames.append(sub)
    else:
        # Single ticker returns flat columns
        sub = df.reset_index()
        sub = sub.rename(columns=lambda x: x.lower())
        sub["ticker"] = tickers[0].strip()
        frames.append(sub)
    if not frames:
        return pd.DataFrame()
    out = pd.concat(frames, ignore_index=True)
    out = out.rename(columns={"index": "date"})
    out = out.dropna(subset=["close"])
    out["date"] = pd.to_datetime(out["date"]).dt.tz_localize("UTC").dt.date
    cols = ["date", "ticker", "open", "high", "low", "close", "volume"]
    return out[cols].sort_values(["ticker", "date"])

In [142]:
fetch_prices(
    tickers=["AAPL", "GOOGL", "MSFT"],
    period_days=30,)

/var/folders/l3/07r79w4x3znd84b94h4223m80000gn/T/ipykernel_56177/1124278083.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


Price,date,ticker,open,high,low,close,volume
0,2025-07-14,AAPL,209.692012,210.670912,207.304722,208.383499,38840100
1,2025-07-15,AAPL,208.982831,211.649803,208.683168,208.872955,42296300
2,2025-07-16,AAPL,210.061609,212.159219,208.403487,209.921768,47490500
3,2025-07-17,AAPL,210.331301,211.559902,209.352401,209.781921,48068100
4,2025-07-18,AAPL,210.630951,211.549907,209.462280,210.940598,48974600
...,...,...,...,...,...,...,...
64,2025-08-07,MSFT,526.799988,528.090027,517.549988,520.840027,16079100
65,2025-08-08,MSFT,522.599976,524.659973,519.409973,522.039978,15531000
66,2025-08-11,MSFT,522.299988,527.590027,519.719971,521.770020,20194400
67,2025-08-12,MSFT,523.750000,530.979980,522.700012,529.239990,18667000
